In [ ]:
# ###
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

# Synthetic dataset (84 days)
np.random.seed(42)
dates = pd.date_range(end=pd.Timestamp.today().normalize(), periods=84, freq='D')
temp = 10 + 4*np.sin(np.linspace(0, 3*np.pi, len(dates))) + np.random.normal(0, 0.5, len(dates))
current = np.abs(np.random.normal(0.25, 0.12, len(dates)))
oxygen = 8.5 + 0.8*np.cos(np.linspace(0, 2*np.pi, len(dates))) + np.random.normal(0, 0.2, len(dates))
biomass = 120 + np.cumsum(np.random.normal(0.4, 0.15, len(dates)))
feed_dose = 0.022*biomass*(1 + 0.08*np.sin(np.linspace(0, 2*np.pi, len(dates))))
waste = np.clip(0.03*feed_dose + 0.6*current + np.random.normal(0, 0.08, len(dates)), 0, None)
growth = np.clip(feed_dose*0.7*(1 + 0.02*(temp-12))*(1 + 0.01*(oxygen-8.5)), 0, None)
fcr = np.where(growth>0, feed_dose / growth, np.nan)
forecast_biomass = pd.Series(biomass).shift(7).fillna(method='bfill') + np.random.normal(0, 1.5, len(dates))
actual_biomass = biomass

df = pd.DataFrame({
    "date": dates,
    "biomass_t": np.round(actual_biomass, 2),
    "feed_dose_t": np.round(feed_dose, 3),
    "feed_waste_t": np.round(waste, 3),
    "FCR": np.round(fcr, 2),
    "temp_C": np.round(temp, 2),
    "current_m_s": np.round(current, 3),
    "oxygen_mg_L": np.round(oxygen, 2),
    "biomass_forecast_t": np.round(forecast_biomass, 2)
})
csv_path = "/mnt/data/scaleaq_feed_analytics_sample.csv"
df.to_csv(csv_path, index=False)

# 2) Charts by matplotlib
# A) FCR rolling
fcr_roll = df.set_index("date")["FCR"].rolling(7, min_periods=3).mean()
plt.figure(figsize=(8,3))
plt.plot(df["date"], fcr_roll)
plt.title("FCR (7-day rolling)"); plt.xlabel("Date"); plt.ylabel("FCR")
fcr_png = "/mnt/data/chart_fcr.png"
plt.tight_layout(); plt.savefig(fcr_png, dpi=200); plt.close()

# B) Waste vs current
x = df["current_m_s"].values; y = df["feed_waste_t"].values
coef = np.polyfit(x, y, 1); xp = np.linspace(x.min(), x.max(), 100); yp = coef[0]*xp + coef[1]
plt.figure(figsize=(5,4))
plt.scatter(x, y, s=12); plt.plot(xp, yp)
plt.title("Feed waste vs. current speed"); plt.xlabel("Current (m/s)"); plt.ylabel("Feed waste (t/day)")
scatter_png = "/mnt/data/chart_waste_current.png"
plt.tight_layout(); plt.savefig(scatter_png, dpi=200); plt.close()

# C) Biomass forecast vs actual
plt.figure(figsize=(8,3))
plt.plot(df["date"], df["biomass_forecast_t"], label="Forecast")
plt.plot(df["date"], df["biomass_t"], label="Actual")
plt.title("Biomass: forecast vs actual"); plt.xlabel("Date"); plt.ylabel("Biomass (t)")
plt.legend()
biomass_png = "/mnt/data/chart_biomass.png"
plt.tight_layout(); plt.savefig(biomass_png, dpi=200); plt.close()

# 3) One-slide PowerPoint
prs = Presentation()
slide = prs.slides.add_slide(prs.slide_layouts[6])

# Title
title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.2), Inches(11), Inches(0.6))
title_tf = title_box.text_frame
title_tf.text = "Feed Optimization Dashboard – Aquaculture (Portfolio Mock)"
title_tf.paragraphs[0].font.size = Pt(24); title_tf.paragraphs[0].font.bold = True

sub_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.8), Inches(11), Inches(0.4))
sub_tf = sub_box.text_frame
sub_tf.text = "Synthetic sample (84 days) • KPIs for last 7 days • Built with Python/PowerPoint"
sub_tf.paragraphs[0].font.size = Pt(12)

# KPI tiles
def add_kpi(x, y, title, value):
    box = slide.shapes.add_textbox(Inches(x), Inches(y), Inches(3.2), Inches(0.9))
    tf = box.text_frame
    p1 = tf.paragraphs[0]; p1.text = title; p1.font.size = Pt(12); p1.font.bold = True
    p2 = tf.add_paragraph(); p2.text = f"{value:.2f}"; p2.font.size = Pt(20); p2.font.bold = True

last7 = df.tail(7)
kpi_fcr = float(last7["FCR"].mean())
kpi_waste = float(last7["feed_waste_t"].sum())
kpi_growth = float(last7["biomass_t"].iloc[-1] - last7["biomass_t"].iloc[0])

add_kpi(0.5, 1.3, "FCR (avg, 7d)", kpi_fcr)
add_kpi(4.0, 1.3, "Feed waste (t, 7d)", kpi_waste)
add_kpi(7.5, 1.3, "Biomass growth (t, 7d)", kpi_growth)

# Place charts
slide.shapes.add_picture(fcr_png, Inches(0.5), Inches(2.3), height=Inches(2.0))
slide.shapes.add_picture(scatter_png, Inches(6.0), Inches(2.3), height=Inches(2.0))
slide.shapes.add_picture(biomass_png, Inches(0.5), Inches(4.6), height=Inches(2.0))

# Footer
foot = slide.shapes.add_textbox(Inches(0.5), Inches(6.8), Inches(11), Inches(0.4))
ft = foot.text_frame
ft.text = "Portfolio mock • Replace with farm data • KPIs: FCR↓, waste↓, growth↑ • Built by Leila Barmoudeh"
ft.paragraphs[0].font.size = Pt(11)

ppt_path = "/mnt/data/ScaleAQ_Portfolio_Dashboard.pptx"
prs.save(ppt_path)

csv_path, fcr_png, scatter_png, biomass_png, ppt_path
